In [50]:
import numpy as np
import pandas as pd
from datetime import datetime

In [124]:
# problem 1

master_trips = pd.read_csv('data/trip_data/trip_data_1.csv', sep=',')

In [125]:

print(master_trips.columns)

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude'],
      dtype='object')


In [126]:
# What's the average trip length? 
distances = master_trips['trip_distance']
mean_dist = np.mean(distances)
median_dist = np.median(distances)
print(median_dist)
print(mean_dist)

1.7
2.770975670679966


In [188]:
master_trips.shape

(14776615, 14)

In [260]:
trips = master_trips.head(200000)

In [261]:
trips[0]

KeyError: 0

In [190]:
unique_meds = trips.medallion.unique()
trips_by_driver = {}
for med in unique_meds:
    hours_dict = {}
    for hour in list(range(24)):
        hours_dict[hour] = {}
    trips_by_driver[med] = hours_dict

In [191]:
for idx, row in trips.iterrows():
    d = datetime.strptime(row.pickup_datetime, '%Y-%m-%d %H:%M:%S')
    ymd = row.pickup_datetime.split(' ')[0]
    if not ymd in trips_by_driver[row.medallion][d.hour]: 
        trips_by_driver[row.medallion][d.hour][ymd] = []
    trips_by_driver[row.medallion][d.hour][ymd].append(row.trip_time_in_secs)
    

In [192]:
# print(trips_by_driver)

In [193]:
# How many trips per hour does a driver have? trips/hour/day/year

# select med from rides whe
# trip_per_driver = 0

# tpd = {
#     med0: {
#         0: {"y1-m1-d1": [1,15,13], "y1-m1-d2", [8,9,10,11]},   -> count[12,15,13,8,9,10,11] / # days
#         1: {"y1-m1-d1": [12,41,12]}
#     }
# }

{
  count:
  unique_days: 
}

# htpd = {
# med0 : { 0: 14, 1: 12}
# }

hourly_trips = {}
for med in trips_by_driver.keys():
    hourly_trips[med] = {}
    for hour in trips_by_driver[med].keys():
        trips_per_day = []
        for day_trips in trips_by_driver[med][hour].values():
            trips_per_day.append(len(day_trips))
        hourly_trips[med][hour] = np.mean(trips_per_day) if len(trips_per_day) else 0
    

In [194]:
print(hourly_trips['89D227B655E5C82AECF13C3F540D4CF4'])

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 1.0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0}


In [195]:

trips_per_hour = {}
for h in range(24):
    trips_per_hour[h] = []
    
for med in hourly_trips.keys():
    for h in hourly_trips[med].keys():
        if hourly_trips[med][h] != 0:
            trips_per_hour[h].append(hourly_trips[med][h])

avg_trips_per_hour = {}
for h in trips_per_hour.keys():
    avg_trips_per_hour[h] = np.mean(trips_per_hour[h]) if len(trips_per_hour[h]) else 0

print(avg_trips_per_hour)

{0: 1.825070821529745, 1: 1.5724907063197027, 2: 1.299906933457422, 3: 1.253158844765343, 4: 1.3050283050283051, 5: 1.353864734299517, 6: 1.8800539083557952, 7: 2.5311077389984824, 8: 2.2811936936936936, 9: 1.5687900906879009, 10: 1.9375768001966085, 11: 2.6488716330987625, 12: 2.78045950336505, 13: 2.6627592425608655, 14: 2.603693813974188, 15: 2.5365256124721602, 16: 2.6988120195667364, 17: 2.707849829351536, 18: 2.639344262295082, 19: 2.4829771371769382, 20: 2.389429763560501, 21: 2.2900371110476736, 22: 2.1914373088685015, 23: 2.208804809052334}


In [196]:
len(hourly_trips.keys()) # unique drivers

7034

In [218]:

# driver_trips = {
#     med0: {
#         'y-m-d': [{s,d}]
#     }
# }

driver_trips = {}

for idx, row in trips.iterrows():
    if row.trip_time_in_secs == 0:
        continue
        
    d = datetime.strptime(row.pickup_datetime, '%Y-%m-%d %H:%M:%S')
    ymd = row.pickup_datetime.split(' ')[0]
    
    if not row.medallion in driver_trips:
        driver_trips[row.medallion] = {}
    
    if not ymd in driver_trips[row.medallion]: 
        driver_trips[row.medallion][ymd] = []
    
    driver_trips[row.medallion][ymd].append({
        'start': d,
        'end': datetime.strptime(row.dropoff_datetime, '%Y-%m-%d %H:%M:%S'),
        'duration': row.trip_time_in_secs
    })
    

In [214]:
print(driver_trips['EA1FC40D75E11B69F59E12B720F201D5'])

[{'2013-01-01': [{'start': datetime.datetime(2013, 1, 1, 15, 11, 48), 'end': datetime.datetime(2013, 1, 1, 15, 18, 10), 'duration': 382}]}, {'2013-01-06': [{'start': datetime.datetime(2013, 1, 6, 0, 18, 35), 'end': datetime.datetime(2013, 1, 6, 0, 22, 54), 'duration': 259}], '2013-01-05': [{'start': datetime.datetime(2013, 1, 5, 18, 49, 41), 'end': datetime.datetime(2013, 1, 5, 18, 54, 23), 'duration': 282}]}, {'2013-01-07': [{'start': datetime.datetime(2013, 1, 7, 23, 54, 15), 'end': datetime.datetime(2013, 1, 7, 23, 58, 20), 'duration': 244}, {'start': datetime.datetime(2013, 1, 7, 23, 25, 3), 'end': datetime.datetime(2013, 1, 7, 23, 34, 24), 'duration': 560}]}, {'2013-01-07': [{'start': datetime.datetime(2013, 1, 7, 15, 27, 48), 'end': datetime.datetime(2013, 1, 7, 15, 38, 37), 'duration': 648}], '2013-01-08': [{'start': datetime.datetime(2013, 1, 8, 7, 51, 16), 'end': datetime.datetime(2013, 1, 8, 7, 51, 20), 'duration': 4}], '2013-01-18': [{'start': datetime.datetime(2013, 1, 18, 

In [219]:

# What's the average utilization of a driver? 
# util = all_trip_durations/(last_start_day - first_start_day).to_s

driver_util_day = {}
for med in driver_trips.keys():
    total_utils = 0
    for day in driver_trips[med].keys():
        ttrips = driver_trips[med][day]
        first = trips[0]
        last = trips[0]
        working_time = 0
        for trip in ttrips:
            working_time += trip['duration']
            if trip['start'] < first['start']:
                first = trip
            if trip['start'] > last['start']:
                last = trip
             
        if last['end'] == first['start']:
            print(med)
            print(first)
            print(last)
        total_time = (last['end'] - first['start']).total_seconds()
        total_utils += float(working_time)/total_time        
    avg_util = total_utils/len(driver_trips[med].keys())
    driver_util_day[med] = avg_util
    

In [222]:
# Problem 2: Pickup matching
# What percent of pickups can potentially be matched with each other? Assume people are willing
# to walk 0.2 miles and wait up to 5 minutes. How does this percentage vary over the course of
# the day? How does this percentage vary as we change the time and distance cutoffs? What are
# the optimal cutoffs?

In [259]:
# group pickups by lat,long ranges, by time

latlons_grid = {}

ACCEPTABLE_PICKUP = 300
ACCEPTABLE_TIME = 5
for idx, row in trips.iterrows():
    scaled_lat = row.pickup_latitude * 1000000
    scaled_lon = row.pickup_longitude * 1000000
    
    rounded_lat = (scaled_lat - (scaled_lat % ACCEPTABLE_PICKUP))/1000000
    rounded_lon = (scaled_lon - (scaled_lon % ACCEPTABLE_PICKUP))/1000000
    
    d = datetime.strptime(row.pickup_datetime, '%Y-%m-%d %H:%M:%S')
    d_minutes = d.hour * 60 + d.minute
    rounded_pickup = d_minutes - (d_minutes % ACCEPTABLE_TIME)
    
    key = str(rounded_lat) + ',' + str(rounded_lon) + ',' + str(rounded_pickup)
    if not key in latlons_grid:
        latlons_grid[key] = []
    latlons_grid[key].append(idx) 
    
# problem 3
# append destination + dropoff time in key and check > 1
    

In [258]:
for k in latlons_grid.keys():
    t = latlons_grid[k]
    if len(t) > 1:
        print("----------------------row-------------------")
        print(k)
        for i in t:
            print(trips.iloc[i].pickup_latitude, trips.iloc[i].pickup_longitude, trips.iloc[i].pickup_datetime)

    

----------------------row-------------------
40.7376,-74.0049,1125
40.737770000000005 -74.004707 2013-01-05 18:49:41
40.737614 -74.004677 2013-01-13 18:45:00
----------------------row-------------------
40.749,-73.9956,590
40.749294 -73.99556 2013-01-08 09:50:05
40.749283 -73.995522 2013-01-14 09:51:00
----------------------row-------------------
40.6485,-73.7835,1420
40.648765999999995 -73.78331 2013-01-10 23:41:51
40.648613 -73.78340899999999 2013-01-13 23:40:00
40.648582 -73.783264 2013-01-13 23:44:00
40.648617 -73.783218 2013-01-13 23:43:00
----------------------row-------------------
40.7499,-73.9914,490
40.750023 -73.99115 2013-01-02 08:12:21
40.750103 -73.991226 2013-01-14 08:13:00
40.750034 -73.991295 2013-01-14 08:11:00
40.750111 -73.991188 2013-01-14 08:10:00
----------------------row-------------------
40.7739,-73.8732,1055
40.773998 -73.873154 2013-01-07 17:38:07
40.774193 -73.87299300000001 2013-01-13 17:38:00
40.774128000000005 -73.8731 2013-01-13 17:37:00
40.774128000000

----------------------row-------------------
40.7478,-73.9893,620
40.747997 -73.989189 2013-01-13 10:23:00
40.747997 -73.989189 2013-01-13 10:22:00
----------------------row-------------------
40.7754,-73.9824,640
40.775428999999995 -73.98221600000001 2013-01-13 10:44:00
40.775688 -73.982292 2013-01-13 10:41:00
----------------------row-------------------
40.7577,-74.001,605
40.757903999999996 -74.00074000000001 2013-01-13 10:05:00
40.757881 -74.000977 2013-01-14 10:05:00
----------------------row-------------------
40.7511,-73.9911,610
40.751324 -73.991051 2013-01-13 10:14:00
40.751251 -73.991096 2013-01-14 10:10:00
----------------------row-------------------
40.7577,-74.001,615
40.757954 -74.000893 2013-01-13 10:18:00
40.757973 -74.000885 2013-01-13 10:17:00
----------------------row-------------------
40.7592,-73.9863,665
40.759266 -73.986137 2013-01-13 11:06:00
40.759209000000006 -73.98603100000001 2013-01-13 11:06:00
----------------------row-------------------
40.7268,-73.98,340

40.7577,-74.001,625
40.7579 -74.000763 2013-01-13 10:27:00
40.757992 -74.00083199999999 2013-01-13 10:27:00
----------------------row-------------------
0.0,0.0,395
0.0 0.0 2013-01-13 06:38:00
0.0 0.0 2013-01-13 06:38:00
0.0 0.0 2013-01-14 06:35:00
0.0 0.0 2013-01-14 06:38:00
0.0 0.0 2013-01-14 06:38:00
0.0 0.0 2013-01-14 06:35:00
0.0 0.0 2013-01-14 06:38:00
0.0 0.0 2013-01-14 06:36:00
0.0 0.0 2013-01-14 06:38:00
0.0 0.0 2013-01-14 06:36:00
0.0 0.0 2013-01-14 06:39:00
0.0 0.0 2013-01-14 06:37:00
0.0 0.0 2013-01-14 06:38:00
0.0 0.0 2013-01-14 06:39:00
----------------------row-------------------
40.746,-73.9749,400
40.746288 -73.974716 2013-01-13 06:43:00
40.746016999999995 -73.974747 2013-01-14 06:41:00
----------------------row-------------------
40.7835,-73.9803,530
40.783794 -73.980164 2013-01-13 08:52:00
40.783596 -73.980049 2013-01-13 08:50:00
40.78376 -73.98009499999999 2013-01-14 08:50:00
----------------------row-------------------
40.7253,-73.9839,210
40.725536 -73.98382600000

0.0 0.0 2013-01-13 09:29:00
0.0 0.0 2013-01-14 09:25:00
0.0 0.0 2013-01-14 09:26:00
0.0 0.0 2013-01-14 09:27:00
0.0 0.0 2013-01-14 09:25:00
0.0 0.0 2013-01-14 09:29:00
0.0 0.0 2013-01-14 09:27:00
0.0 0.0 2013-01-14 09:27:00
0.0 0.0 2013-01-14 09:26:00
----------------------row-------------------
40.6485,-73.7838,560
40.648773 -73.783592 2013-01-13 09:20:00
40.648635999999996 -73.783585 2013-01-13 09:23:00
----------------------row-------------------
40.7631,-73.9785,430
40.763268 -73.9785 2013-01-13 07:14:00
40.763168 -73.978218 2013-01-13 07:12:00
----------------------row-------------------
40.7379,-74.0004,405
40.738087 -74.000206 2013-01-13 06:48:00
40.737988 -74.000259 2013-01-14 06:45:00
----------------------row-------------------
40.7304,-74.0004,420
40.730576 -74.000359 2013-01-13 07:00:00
40.730507 -74.000229 2013-01-13 07:00:00
----------------------row-------------------
40.7298,-73.9917,405
40.729801 -73.991493 2013-01-13 06:48:00
40.729828000000005 -73.991646 2013-01-14 0

----------------------row-------------------
40.7328,-73.9959,480
40.73299 -73.995888 2013-01-13 08:04:00
40.732979 -73.995773 2013-01-14 08:04:00
40.733028000000004 -73.995674 2013-01-14 08:04:00
----------------------row-------------------
40.7517,-73.9944,435
40.751942 -73.994225 2013-01-13 07:17:00
40.7519 -73.99413299999999 2013-01-14 07:15:00
----------------------row-------------------
40.7787,-73.9563,570
40.778824 -73.956017 2013-01-13 09:34:00
40.778965 -73.956017 2013-01-13 09:34:00
----------------------row-------------------
0.0,0.0,430
0.0 0.0 2013-01-13 07:12:00
0.0 0.0 2013-01-01 07:14:00
0.0 0.0 2013-01-13 07:12:00
0.0 0.0 2013-01-13 07:10:00
0.0 0.0 2013-01-14 07:12:00
0.0 0.0 2013-01-14 07:12:00
0.0 0.0 2013-01-14 07:12:00
0.0 0.0 2013-01-14 07:11:00
0.0 0.0 2013-01-14 07:12:00
0.0 0.0 2013-01-14 07:14:00
0.0 0.0 2013-01-14 07:13:00
0.0 0.0 2013-01-14 07:13:00
0.0 0.0 2013-01-14 07:11:00
0.0 0.0 2013-01-14 07:11:00
0.0 0.0 2013-01-14 07:10:00
0.0 0.0 2013-01-14 07:14

...

40.753456 -73.97776 2013-01-14 08:29:00
----------------------row-------------------
40.7367,-73.989,550
40.736946 -73.988831 2013-01-14 09:12:00
40.736732 -73.988922 2013-01-14 09:12:00
----------------------row-------------------
40.7016,-74.0127,555
40.701855 -74.012505 2013-01-14 09:17:00
40.701733000000004 -74.012695 2013-01-14 09:15:00
----------------------row-------------------
40.7625,-73.9821,495
40.762737 -73.981812 2013-01-14 08:16:00
40.762707 -73.981888 2013-01-14 08:19:00
40.762691 -73.981995 2013-01-14 08:15:00
40.762791 -73.981903 2013-01-14 08:15:00
----------------------row-------------------
40.7553,-73.9773,500
40.755363 -73.977058 2013-01-14 08:20:00
40.755592 -73.97728000000001 2013-01-14 08:24:00
----------------------row-------------------
40.7697,-73.8642,475
40.769829 -73.863998 2013-01-14 07:55:00
40.76976 -73.86395999999999 2013-01-14 07:58:00
----------------------row-------------------
40.7376,-73.9803,500
40.737606 -73.98024699999999 2013-01-14 08:24:00


40.784248 -73.95639 2013-01-14 07:45:00
----------------------row-------------------
40.7379,-73.9881,460
40.738056 -73.988007 2013-01-14 07:41:00
40.738091 -73.98793 2013-01-14 07:44:00
----------------------row-------------------
40.7442,-73.9959,475
40.744251 -73.995796 2013-01-14 07:55:00
40.744244 -73.995743 2013-01-14 07:55:00
----------------------row-------------------
40.7562,-73.9902,475
40.756283 -73.990051 2013-01-14 07:55:00
40.75629 -73.990143 2013-01-14 07:57:00
40.756279 -73.98999 2013-01-14 07:56:00
----------------------row-------------------
40.7625,-73.9785,455
40.762516 -73.978409 2013-01-14 07:37:00
40.762539000000004 -73.97827099999999 2013-01-14 07:39:00
----------------------row-------------------
40.7562,-73.9902,465
40.756271000000005 -73.990021 2013-01-14 07:45:00
40.756316999999996 -73.989937 2013-01-14 07:48:00
40.756439 -73.99006700000001 2013-01-14 07:46:00
----------------------row-------------------
40.7343,-73.9836,465
40.734402 -73.98336 2013-01-14 0

40.64529 -73.776657 2013-01-14 08:38:00
40.645428 -73.77669499999999 2013-01-14 08:36:00
----------------------row-------------------
40.7676,-73.9644,555
40.767899 -73.96414200000001 2013-01-14 09:18:00
40.767662 -73.964104 2013-01-14 09:17:00
----------------------row-------------------
40.6452,-73.7769,465
40.645432 -73.776634 2013-01-14 07:49:00
40.645393 -73.776718 2013-01-14 07:46:00
40.645481 -73.77668800000001 2013-01-14 07:49:00
----------------------row-------------------
40.7715,-73.9869,520
40.771648 -73.98677099999999 2013-01-14 08:40:00
40.771534 -73.98671 2013-01-14 08:43:00
----------------------row-------------------
40.7508,-73.9941,555
40.750961 -73.994049 2013-01-14 09:16:00
40.751026 -73.994057 2013-01-14 09:16:00
----------------------row-------------------
40.7475,-73.9809,510
40.747593 -73.98075899999999 2013-01-14 08:31:00
40.747681 -73.980774 2013-01-14 08:31:00
----------------------row-------------------
40.7526,-73.9671,565
40.752792 -73.966957 2013-01-14 0

40.752,-73.9782,515
40.752224 -73.977982 2013-01-14 08:39:00
40.752235 -73.978065 2013-01-14 08:39:00
40.752136 -73.978065 2013-01-14 08:35:00
----------------------row-------------------
40.7616,-73.9824,520
40.761707 -73.982262 2013-01-14 08:44:00
40.761833 -73.982254 2013-01-14 08:42:00
----------------------row-------------------
40.7415,-74.0013,465
40.741684 -74.001053 2013-01-14 07:48:00
40.741539 -74.00123599999999 2013-01-14 07:49:00
----------------------row-------------------
40.725,-73.9956,505
40.725097999999996 -73.995399 2013-01-14 08:28:00
40.725182000000004 -73.995308 2013-01-14 08:28:00
----------------------row-------------------
40.755,-73.9653,510
40.755234 -73.965233 2013-01-14 08:32:00
40.755142 -73.965225 2013-01-14 08:34:00
40.755245 -73.965126 2013-01-14 08:34:00
----------------------row-------------------
40.7565,-73.9902,515
40.756721 -73.990166 2013-01-14 08:35:00
40.756657000000004 -73.990173 2013-01-14 08:37:00
----------------------row------------------

----------------------row-------------------
40.7478,-73.9893,515
40.747944 -73.989189 2013-01-14 08:39:00
40.747952000000005 -73.98925 2013-01-14 08:38:00
----------------------row-------------------
40.7688,-73.9611,505
40.769058 -73.96109 2013-01-14 08:28:00
40.768986 -73.960808 2013-01-14 08:27:00
----------------------row-------------------
40.7523,-73.9782,525
40.752388 -73.97811899999999 2013-01-14 08:49:00
40.752369 -73.977913 2013-01-14 08:48:00
----------------------row-------------------
40.7658,-73.9548,650
40.76593 -73.954544 2013-01-14 10:51:00
40.7658 -73.95475 2013-01-14 10:50:00
----------------------row-------------------
40.7343,-73.9992,650
40.734367 -73.999084 2013-01-14 10:52:00
40.734413 -73.999084 2013-01-14 10:54:00
----------------------row-------------------
40.7349,-73.9986,530
40.735069 -73.998566 2013-01-14 08:54:00
40.735195000000004 -73.998482 2013-01-14 08:50:00
----------------------row-------------------
40.7523,-73.9896,510
40.75251 -73.989395 2013-0